In [16]:
from google.colab import drive
drive.mount('/content/drive')

data_root='/content/drive/MyDrive/Semester 5/PSC1_M. Ilham Rigan Agachi/Nyoba'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import librari

In [2]:
!pip install nltk tensorflow ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.5 MB/s eta 0:00:00


In [17]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import tensorflow as tf # Make sure to import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Pra-pemrosesan Data

In [18]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

try:
    data_file = open(data_root + '/sholatislam.json').read()
    intents = json.loads(data_file)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Please check sholatislam.json around line {e.lineno}, column {e.colno}, position {e.pos}")
    print("The file might contain invalid characters or be malformed.")
    # Handle the error appropriately, e.g., exit the program or try to load a backup.
except FileNotFoundError:
    print(f"The file sholatislam.json was not found. Please check if the path is correct.")

# melihat isi dari data json / intents

In [19]:
intents

{'intents': [{'tag': 'dialog.introduction',
   'patterns': ['hai, perkenalkan aku adalah %name%',
    'hai nama aku adalah %name%',
    'perkenalkan nama aku %name%',
    'lo siapa'],
   'responses': ['Hai, aku adalah Tumabot',
    'Hai aku Tumabot Salam kenal.',
    'perkenalkan aku Tumabot, senang mengenal kamu :)',
    'Hallo, aku Tumabot, apa ada yang bisa saya bantu?']},
  {'tag': 'greeting',
   'patterns': ["Assalamu'alaikum",
    'Assalamualaikum',
    'assalamu',
    'assalamulaikum',
    'Halo',
    'Hi',
    'Hai',
    'hai',
    'Hello',
    'Salam sejahtera',
    'Selamat pagi!',
    'Apa kabar?',
    'Hai, ada yang bisa dibantu?',
    'Assalamualaikum, boleh tanya?',
    'Selamat sore, ada informasi?'],
   'responses': ["Wa'alaikumussalam! Ada yang bisa saya bantu?",
    'waalikumsalam sobat! Ada yang bisa saya bantu?',
    'waalikumsalam teman! tanya tumabot?',
    'Halo! Saya TumaBot silakan bertanya.',
    'Halo! Tumabot siap membantu',
    'Assalamualaikum, bagaimana s

# tokenize / membuat sekelompok kata

In [20]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [21]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# mengelompokkan dokumen

In [22]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


814 documents
109 classes ['ajaran.imamsyafii', 'ajaran.rasul', 'bacaan_al_fatihah', 'bacaan_ruku', 'bacaan_salam', 'bacaan_sujud', 'bacaan_surah_pendek', 'bacaan_tasyahud_akhir', 'bacaan_tasyahud_awal', 'deteksi.gerakan.sholat', 'developer', 'developer.tumanina', 'dialog.FiturWeb', 'dialog.holdon', 'dialog.idontcare', 'dialog.introduction', 'dialog.kabar', 'dialog.sorry', 'dialog.thank', 'doa', 'doa.setelah.sholat', 'doa_qunut', 'doa_setelah_salam', 'download.tumanina', 'duduk_di_antara_dua_sujud', 'duduk_iftirosy', 'duduk_tasyahud_awal', 'duduk_tawarruk', 'fitur.alqurandigital', 'fitur.artikelkalam', 'fitur.belajarsholat', 'fitur.doaharian', 'fitur.kiblat', 'fitur.pantausholat', 'fitur.tasbih', 'fitur.tumabot', 'fitur.tumanina', 'fitur.waktusholat', 'gerakan.sholat', 'goodbye', 'greeting', 'greetings.afternoon', 'greetings.bye', 'greetings.day', 'greetings.hello', 'greetings.howareyou', 'greetings.kenalan', 'greetings.nicetomeetyou', 'greetings.nicetoseeyou', 'greetings.nicetotalktoy

# training data

In [23]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
# Shuffle our features and ensure the training data stays as a list of lists
random.shuffle(training)

# Create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]  # Extracting the 'bag' part
train_y = [item[1] for item in training]  # Extracting the 'output_row' part

# Optionally, convert train_x and train_y to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")


Training data created


# Model CNN sgd

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential()
model.add(Dense(64, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
hist = model.fit(train_x, train_y, epochs=100, batch_size=8, verbose=1, callbacks=[early_stopping])

# Save the model
model.save('chatbot_model.h5')
print("Model created and saved")

Epoch 1/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0153 - loss: 4.6870
Epoch 2/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0681 - loss: 4.5625
Epoch 3/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0520 - loss: 4.2947
Epoch 4/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1108 - loss: 3.9365
Epoch 5/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1995 - loss: 3.6134
Epoch 6/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2236 - loss: 3.3135
Epoch 7/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2533 - loss: 3.0942
Epoch 8/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3369 - loss: 2.6956
Epoch 9/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3565 - loss: 2.4983
Epoch 10/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4190 - loss: 2.3432
Epoch 11/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4426 - loss: 2.1574
Epoch 12/100
102/102 ━━━━━━━━━━━━━━━━━━━━

Model created and saved


# import pickle & h5

In [25]:
from tensorflow.keras.models import load_model
import pickle
import numpy as np

# Load model
model = load_model('chatbot_model.h5')

# Load supporting data
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Test a sample input
def bow(sentence, words):
    sentence_words = sentence.split()  # Simple tokenization
    bag = [1 if w in sentence_words else 0 for w in words]
    return np.array(bag)

# Test a prediction
sentence = ""
bag = bow(sentence, words)
prediction = model.predict(np.array([bag]))
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Prediction: [[2.27756143e-04 3.31579213e-05 1.37628303e-04 1.52977445e-04
  2.83171230e-05 2.15370655e-05 2.65350900e-05 1.76925569e-05
  2.61597761e-05 2.63997435e-07 1.99995593e-05 5.37753886e-06
  2.32518394e-03 1.09619992e-02 1.66645594e-04 5.30718244e-05
  7.47130904e-03 1.67783393e-04 5.58870938e-03 1.64473546e-03
  1.99452643e-06 1.04242144e-03 7.23646963e-06 3.44986802e-05
  5.89160209e-06 4.57977003e-05 6.40233120e-05 8.02619161e-06
  3.89159628e-04 1.84641147e-04 2.92971818e-04 2.54230830e-03
  3.12560500e-04 9.92301138e-05 4.27883788e-04 1.88385148e-03
  1.79333369e-07 2.73910118e-05 9.53305207e-05 3.81896738e-03
  5.08008301e-01 3.17316735e-04 4.83515579e-03 1.87058803e-02
  5.56962043e-02 1.14019886e-02 1.02370651e-03 3.22712949e-05
  2.07584253e-05 6.58392764e-07 2.62570044e-04 7.18443262e-05
  5.54484501e-03 3.68831752e-05 2.01372518e-06 3.65070518e-05
  1.92197755e-01 1.46190330e-01 1.14852905e-04 1.81165722e-03
  3.88243265e-04 4.4

In [26]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [27]:
import ipywidgets as widgets
from IPython.display import display

def send_message(sender):
    msg = input_box.value
    input_box.value = ''

    if msg != '':
        chat_log.append('You: ' + msg)
        res = chatbot_response(msg)
        chat_log.append('Bot: ' + res)

        # Display the updated chat log
        output_area.clear_output(wait=True)
        with output_area:
            for line in chat_log:
                print(line)

# Create UI elements
input_box = widgets.Textarea(value='', placeholder='Type here...', layout={'height': '50px', 'width': '300px'})
send_button = widgets.Button(description='Send', layout={'width': '80px'})
output_area = widgets.Output(layout={'height': '300px'})

# Bind the function to the button click event
send_button.on_click(send_message)

# Display UI
display(widgets.VBox([input_box, send_button, output_area]))

# Initialize chat log
chat_log = []


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
